In [211]:
#imports
import numpy as np
import pandas as pd
import os.path
import pprint as pp
from sklearn import ensemble
import datetime

In [2]:
#get data
pathToFiles = "..\\data_preperation\\" # windows path, escaped backslash
folderCleaned = "cleaned\\"

games = pd.read_pickle(pathToFiles + folderCleaned + "merge_all_game-data.pkl")
games_data = pd.read_pickle(pathToFiles + folderCleaned + "merge_all_games.pkl")

In [3]:
#join the two game datasets
games_with_reset_index = games.reset_index(level= 1)
full_games = games_with_reset_index.join(games_data, how = 'right', lsuffix= "_")

In [4]:
availableMoves =pd.Series(full_games.available_moves)

In [73]:
# reshape the field in 9,9 for human convenience
newField = []
newAvailable_Moves = []
for feld in full_games.field:
    x = feld.reshape(9,9)
    newField.append(x)
fieldSeries = pd.Series(newField)

for j in range(0, len(availableMoves)):# manchmal steht da nur ein leeres objekt weil es beim letzen zug keine
    if (isinstance(availableMoves.iloc[j], np.ndarray)):
        x = availableMoves.iloc[j].reshape(9,9)
        newAvailable_Moves.append(x)
    else:
        newAvailable_Moves.append(np.NaN)
availableSeries = pd.Series(newAvailable_Moves)

In [202]:
#erstellt ein 3x3 macroboard, erst die funktionen aktivieren
macroBoards = []
for k in range(0, len(availableMoves)):
    if (isinstance(availableMoves.iloc[k], np.ndarray)):
        macroBoard = feature_MicroboardWinningMove(field= fieldSeries.iloc[k], _availableMoves= availableSeries.iloc[k])
        macroBoards.append(macroBoard)
    else:
        macroBoards.append(np.NaN)
full_games['MacroBoard'] = macroBoards

In [194]:
# erstellt ein 3x3 macroboard aus den derzeitigen daten
def microBoardWon(vonX, bisX, vonY, bisY, macroX, macroY, _availableMoves, macroBoard):
    if (_availableMoves[vonX:bisX, vonY:bisY] == 17).any(): #wenn irgendwo eine 17 in diesem microboard steht hat player 1 das micro gewonnen
        macroBoard[macroX, macroY] = 1
    elif (_availableMoves[vonX:bisY, vonX:bisY] == 33).any():# same für player 2 mit 33
        macroBoard[macroX, macroY] = 2
    else:
        macroBoard[macroX, macroY] = 0
def feature_MicroboardWinningMove(field, _availableMoves):
    macroBoard = np.zeros((3,3))
    fieldIsNotPlayable = np.NaN
    winsMicroboardPlayer1 = "1"
    winsMicroboardPlayer2 = "2"
    winsNoOne = "0"
    # ich hätte das gerne in der schleife aber ich weiß nicht genau wie, deswegen hir die gleiche version 9 mal
    microBoardWon(0,3,0,3, 0,0,_availableMoves,macroBoard)
    microBoardWon(0,3,3,6, 0,1,_availableMoves,macroBoard)
    microBoardWon(0,3,6,9, 0,2,_availableMoves,macroBoard)
    microBoardWon(3,6,0,3, 1,0,_availableMoves,macroBoard)
    microBoardWon(3,6,3,6, 1,1,_availableMoves,macroBoard)
    microBoardWon(3,6,6,9, 1,2,_availableMoves,macroBoard)
    microBoardWon(6,9,0,3, 2,0,_availableMoves,macroBoard)
    microBoardWon(6,9,3,6, 2,1,_availableMoves,macroBoard)
    microBoardWon(6,9,6,9, 2,2,_availableMoves,macroBoard)
    return macroBoard


In [212]:
# ich speichere das jetzt einfahc mal als pickle, und öffne dann ein neues programm zum featuere extrakten
date = datetime.datetime.now()
full_games.to_pickle(str(date.date()) + '_joinedGames.pkl')